In [1]:
from queue import Queue

In [109]:
class Module:
    def __init__(self, destinations: list[str]):
        self.destinations = destinations

    def process(self, pulse: bool, parent: str) -> bool | None:
        pass

class Broadcast(Module):
    def process(self, pulse: bool, parent: str = '') -> bool:
        return pulse
    
class FlipFlop(Module):
    def __init__(self, destinations: list[str]):
        super().__init__(destinations)
        self.state = False
    def process(self, pulse: bool, parent: str = '') -> bool | None:
        if pulse:
            return None
        if self.state == True:
            self.state = False
            return False
        else:
            self.state = True
            return True
    
class Conjunction(Module):
    def __init__(self, destinations: list[str], inputs: int):
        super().__init__(destinations)
        self.memory: dict[str, bool] = {}
        self.inputs = inputs
    def process(self, pulse: bool, parent: str) -> bool | None:
        self.memory[parent] = pulse
        return not((len(self.memory) == self.inputs) and (all(self.memory.values())))


In [45]:
def parse(text: list[str]) -> dict[str, Module]:
    d: dict[str, Module] = {}
    for l in text:
        tag, dest = l.strip().split(' -> ')
        dl = dest.split(', ')
        if tag[0] == '%':
            m = FlipFlop(dl)
            tag = tag[1:]
        elif tag[0] == '&':
            tag = tag[1:]
            ninp = sum(1 for l1 in text if ' '+tag in l1)
            m = Conjunction(dl, ninp)
        else:
            m = Broadcast(dl)
        d |= {tag: m}
    return d

In [34]:
with open('test1.txt', 'rt') as f:
    test1 = f.readlines()

In [119]:
mconf = parse(test1)

In [118]:
def press_button(mconf: dict[str, Module], debug: bool = False) -> tuple[int, int]:
    q = Queue()
    q.put(('button', False, 'broadcaster'))
    low = 1
    high = 0
    while not q.empty():
        pulse = q.get()
        if pulse[2] not in mconf.keys():
            continue
        mod = mconf[pulse[2]]
        processed = mod.process(pulse[1], pulse[0])
        if processed is None:
            continue
        if processed:
            high += len(mod.destinations)
        else:
            low += len(mod.destinations)
        for it in mod.destinations:
            q.put((pulse[2], processed, it))
            if debug:
                print((pulse[2], processed, it))
    return low, high

In [120]:
press_button(mconf, True)

('broadcaster', False, 'a')
('broadcaster', False, 'b')
('broadcaster', False, 'c')
('a', True, 'b')
('b', True, 'c')
('c', True, 'inv')
('inv', False, 'a')
('a', False, 'b')
('b', False, 'c')
('c', False, 'inv')
('inv', True, 'a')


(8, 4)

In [121]:
with open('test2.txt', 'rt') as f:
    test2 = f.readlines()

mconf2 = parse(test2)

In [122]:
press_button(mconf2, True)

('broadcaster', False, 'a')
('a', True, 'inv')
('a', True, 'con')
('inv', False, 'b')
('con', True, 'output')
('b', True, 'con')
('con', False, 'output')


(4, 4)

In [123]:
press_button(mconf2, True)

('broadcaster', False, 'a')
('a', False, 'inv')
('a', False, 'con')
('inv', True, 'b')
('con', True, 'output')


(4, 2)

In [124]:
press_button(mconf2, True)

('broadcaster', False, 'a')
('a', True, 'inv')
('a', True, 'con')
('inv', False, 'b')
('con', False, 'output')
('b', False, 'con')
('con', True, 'output')


(5, 3)

In [125]:
def part1(test: list[str]) -> int:
    mconf = parse(test)
    low = 0
    high = 0
    for _ in range(1000):
        l, h = press_button(mconf)
        low += l
        high += h
    return low*high

In [126]:
part1(test1)

32000000

In [127]:
part1(test2)

11687500

In [128]:
with open('input', 'rt') as f:
    inp = f.readlines()

In [129]:
with open('output1', 'wt') as f:
    f.write(str(part1(inp)))